# YOLOv8 Model Predictions

In [1]:
import os
import cv2
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
from ultralytics import YOLO
from IPython.display import display, Image

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils

palette0 = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000']) # Okabe-Ito
palette = palette0
display(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0) # very good to see

[(0.9019607843137255, 0.6235294117647059, 0.0),
 (0.33725490196078434, 0.7058823529411765, 0.9137254901960784),
 (0.0, 0.6196078431372549, 0.45098039215686275),
 (0.9411764705882353, 0.8941176470588236, 0.25882352941176473),
 (0.0, 0.4470588235294118, 0.6980392156862745),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.4745098039215686, 0.6549019607843137),
 (0.0, 0.0, 0.0)]

In [2]:
!nvidia-smi
num_gpus = torch.cuda.device_count()
for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Thu Jun 13 03:47:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:01:00.0 Off |                  Off |
| 30%   27C    P8              28W / 300W |  11208MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load YOLOv8 model

In [3]:
# Select model for prediction
# model_dirname = "umineko-2024-v7i-yolov8"
model_dirname = "umineko-2024-v8i-yolov8"

# Load a model
WEIGHTS_PATH = "yolov8s-seg.pt"
model = YOLO(WEIGHTS_PATH)  # load an official model
model = YOLO(f"/mnt/nfs_p/yolo/{model_dirname}/output/train/runs/seed-00/weights/best.pt")  # load a custom model

## Process sequential images (test)

In [ ]:
BASE_DIR = "/mnt/nfs_p/yolo/data/extracted-frames/umineko-2024/"
os.chdir(BASE_DIR)
current_directory = os.getcwd()
print(current_directory)

In [ ]:
data_dir_list = sorted(glob.glob(f"{BASE_DIR}/*/*"))
print(f"{len(data_dir_list)} videos")

In [ ]:
for i, data_dir in enumerate(data_dir_list):
    image_path_list = sorted(glob.glob(f"{data_dir}/*.jpg"))
    session_id = os.path.basename(data_dir)
    print(f"{i:02} | session_id: {session_id} | num frames: {len(image_path_list)}")

## Run inference

In [ ]:
mask_color = np.array([167, 121, 204])  # pink

# DEVICE = 'cpu'
DEVICE = 'cuda:0'
# DEVICE = 'cuda:1'

# for j, data_dir in enumerate(data_dir_list[:1]):
for j, data_dir in enumerate(data_dir_list[1:]):
# for j, data_dir in enumerate(data_dir_list):
    image_path_list = sorted(glob.glob(f"{data_dir}/*.jpg"))
    session_id = os.path.basename(data_dir)
    print(f"{j:02} | session_id: {session_id} | num frames: {len(image_path_list)}")
    
    threshold = 0.5
    null_mask = np.zeros((480, 640), dtype=np.float32)
    prev_mask = null_mask
    # print(prev_mask.shape)
    pixel_count_list = []
    diff_pixel_count_list = []
    # for i, image_path in enumerate(image_path_list[:20]):
    # for i, image_path in enumerate(image_path_list[1930:2000]):
    for i, image_path in tqdm(enumerate(image_path_list), total=len(image_path_list)):
        image = cv2.imread(image_path)
        results = model.predict(image_path, save=False, imgsz=640, conf=0.5, device=DEVICE)
        # show_masked_image = True
        show_masked_image = True if i % 1000 == 0 else False
        curr_mask, pixel_count, fig = utils.create_masked_image(
            results, image, mask_color=mask_color, alpha=0.5, show=show_masked_image
        )
        pixel_count_list.append(pixel_count)
        # print(curr_mask.dtype)
        # print(curr_mask.shape)
        
        # Calculate difference
        diff_mask = cv2.absdiff(curr_mask, prev_mask)
        diff_pixel_count = 0 if i == 0 else np.sum(diff_mask > threshold) 
        diff_pixel_count_list.append(diff_pixel_count)
        prev_mask = curr_mask # update previous mask
    
    total_pixel_count = 480 * 640
    data_dict = {
        'pixel_count': pixel_count_list,
        'absdiff_pixel_count': diff_pixel_count_list,
    }
    df = pd.DataFrame(data_dict)
    df.insert(len(df.columns), 'pixel_count_p', df['pixel_count']/total_pixel_count)
    df.insert(len(df.columns), 'absdiff_pixel_count_p', df['absdiff_pixel_count']/total_pixel_count)
    save_path = f"/mnt/nfs_p/yolo/output/{model_dirname}/predicted-data/{session_id}.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    df.to_csv(save_path, index=False)

## Visualize 

In [ ]:
df_vis = pd.read_csv("/mnt/nfs_p/yolo/output/predicted-data/LBP00_S00.csv")
display(df)

In [ ]:
display(df[1936:2000].head(10))

In [ ]:
GRIDSPEC_KW = {'wspace': 0.3, 'hspace': 0.3}
fig, axes = plt.subplots(2, 1, figsize=(20, 10), gridspec_kw=GRIDSPEC_KW)
ax_list = axes.flatten().tolist()
t = np.arange(0, len(df), 1)
y_list = ['pixel_count_p', 'absdiff_pixel_count_p']
for a, ax in enumerate(ax_list):
    ax = sns.lineplot(
        ax=ax, x=t, y=df[f"{y_list[a]}"], 
        color=palette0[a], label=f"{y_list[a]}"
    )
    ax.grid(which='major')
    ax.set_xlim(1500 - 10, 2500 + 10)
    ax.set_ylabel(y_list[a], labelpad=20)
    ax.set_yticks(np.arange(-0.2, 1.2, 0.2))
    ax.set_ylim(-0.1, 1.1)
    
plt.show()
plt.close()